In [12]:
import json
import requests
from urllib.parse import quote
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

SPOTIPY_CLIENT_ID = "your_spotify_client_id"
SPOTIPY_CLIENT_SECRET = "your_spotify_client_secret"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET
))

playlist_link = "your_spotify_playlist_link"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

results = sp.playlist_tracks(playlist_URI)
tracks = []

for item in results['items']:
    track = item['track']
    if track is None: continue
    name = track['name']
    artist = track['artists'][0]['name']
    tracks.append({'name': name, 'artist': artist})

print(f"🎧 Extracted {len(tracks)} tracks from Spotify.\n")

apple_links = []
errors = 0

for t in tracks:
    name, artist = t['name'], t['artist']

    if not name or not artist:
        print(f"Skipping invalid track: {t}")
        continue

    query = quote(f"{name} {artist}")
    url = f"https://itunes.apple.com/search?term={query}&media=music&limit=1"

    try:
        response = requests.get(url)
        data = response.json()
        if data['resultCount'] > 0:
            track_url = data['results'][0]['trackViewUrl']
            print(f"Found on Apple Music: {name} by {artist}")
            apple_links.append({
                'name': name,
                'artist': artist,
                'apple_music_url': track_url
            })
        else:
            print(f"Not found: {name} by {artist}")
            errors += 1
    except Exception as e:
        print(f"API error: {e}")
        errors += 1

with open("apple_music_links.json", "w") as f:
    json.dump(apple_links, f, indent=2)

with open("apple_music_urls.txt", "w") as f:
    for track in apple_links:
        f.write(track['apple_music_url'] + "\n")

print(f"\n Done! Found {len(apple_links)} Apple Music links, {errors} errors.")
print(" Saved to 'apple_music_links.json' and 'apple_music_urls.txt'")

from google.colab import files

files.download("apple_music_links.json")
files.download("apple_music_urls.txt")



🎧 Extracted 21 tracks from Spotify.

Found on Apple Music: Best Part of Me (feat. YEBBA) by Ed Sheeran
Found on Apple Music: Perfect by Ed Sheeran
Found on Apple Music: You Are The Reason by Calum Scott
Found on Apple Music: Make You Feel My Love by Adele
Found on Apple Music: Tennessee Whiskey by Chris Stapleton
Found on Apple Music: Can't Help Falling in Love by Elvis Presley
Found on Apple Music: Falling In Love by Cigarettes After Sex
Found on Apple Music: The Night We Met by Lord Huron
Found on Apple Music: Sway by Michael Bublé
Found on Apple Music: Everything by Michael Bublé
Found on Apple Music: Photograph by Ed Sheeran
Found on Apple Music: The Joker And The Queen (feat. Taylor Swift) by Ed Sheeran
Found on Apple Music: Until I Found You by Stephen Sanchez
Found on Apple Music: Yellow by Coldplay
Found on Apple Music: Magical by Ed Sheeran
Found on Apple Music: Love by Lana Del Rey
Found on Apple Music: This Town by Niall Horan
Found on Apple Music: If We Have Each Other by A

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>